**IMPORT LIBRARIES**

In [1]:
from warnings import filterwarnings

filterwarnings('ignore')

In [2]:
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification, TFAutoModelForSeq2SeqLM, TFBertLMHeadModel, pipeline

In [3]:
num = 3

**IMPORT AND DOWNLOAD MODELS**

In [4]:
rating_classification_path = ['bert-base-uncased', './modelos/rating-classification/weights/']
yelp_classification_path = ['roberta-base', './modelos/yelp-classification/weights/']
google_response_path = ['facebook/bart-large', './modelos/review-response/weights/']

In [5]:
rating_class = [AutoTokenizer.from_pretrained(rating_classification_path[0]), 
                TFAutoModelForSequenceClassification.from_pretrained(rating_classification_path[1], num_labels=5)]

yelp_class = [AutoTokenizer.from_pretrained(yelp_classification_path[0]), 
              TFAutoModelForSequenceClassification.from_pretrained(yelp_classification_path[1])]

google_response = [AutoTokenizer.from_pretrained(google_response_path[0]), 
                TFAutoModelForSeq2SeqLM.from_pretrained(google_response_path[1])]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./modelos/rating-classification/weights/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at ./modelos/yelp-classification/weights/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at ./mod

In [6]:
from datasets import load_dataset

In [7]:
data = load_dataset('csv', data_files='./data/google/reviews/washington.csv', split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/camat/.cache/huggingface/datasets/csv/default-af9ae02c03c907df/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


In [8]:
data = data.filter(lambda example: example['text'] is not None)

Filter:   0%|          | 0/1289220 [00:00<?, ? examples/s]

<hr>

**REVIEW COMO EJEMPLO PARA LOS MODELOS**

In [24]:
review_example = data[11223]['text']
review_example

"Its not the money,  it's the character of the team as a whole. Most of everyone is happy go luck here. There's grumbles anywhere I've gone to but grumbles here are 5inefficient things that you/we have the capacity to change. (Theses items are my only neutral or critical statements . I've been working since I was 8 yrs old and this is easily the best employer I've ever been able to earn with.\nThanks,to the owners, lower management , the greatest teams and HR. You've all been a blessing in the beginning of my young baby girl and immediate family's life and I am truly grateful for the opportunity to be here."

<hr>

**GOOGLE RESPONSE**

In [25]:
inputs = google_response[0](review_example, return_tensors='tf')
output = google_response[1].generate(**inputs, max_new_tokens=500)
decoded = google_response[0].decode(output[0], skip_special_tokens=True)

print('<REVIEW>')
print(review_example)
print()
print('<OUTPUT>')
print(decoded)

<REVIEW>
Its not the money,  it's the character of the team as a whole. Most of everyone is happy go luck here. There's grumbles anywhere I've gone to but grumbles here are 5inefficient things that you/we have the capacity to change. (Theses items are my only neutral or critical statements . I've been working since I was 8 yrs old and this is easily the best employer I've ever been able to earn with.
Thanks,to the owners, lower management , the greatest teams and HR. You've all been a blessing in the beginning of my young baby girl and immediate family's life and I am truly grateful for the opportunity to be here.

<OUTPUT>
Thank you so much for taking the time to leave us such a wonderful review! We are so happy to hear that you have had such a positive experience with us and that you are happy with your experience. We appreciate your business and look forward to continuing to serve you and your family for years to come!


<hr>

**YELP CLASSIFICATION**

In [26]:
labels = ['cool', 'funny', 'useful']

input_data = yelp_class[0](review_example, return_tensors='tf')
output = yelp_class[1](**input_data)
decoded = labels[tf.argmax(output.logits, axis=1).numpy()[0]]

print('<REVIEW>')
print(review_example)
print()
print('<OUTPUT>')
print(decoded)

<REVIEW>
Its not the money,  it's the character of the team as a whole. Most of everyone is happy go luck here. There's grumbles anywhere I've gone to but grumbles here are 5inefficient things that you/we have the capacity to change. (Theses items are my only neutral or critical statements . I've been working since I was 8 yrs old and this is easily the best employer I've ever been able to earn with.
Thanks,to the owners, lower management , the greatest teams and HR. You've all been a blessing in the beginning of my young baby girl and immediate family's life and I am truly grateful for the opportunity to be here.

<OUTPUT>
useful


<hr>

**RATINGS CLASSIFICATION**

In [27]:
inputs = rating_class[0](review_example, return_tensors='tf')
output = rating_class[1](**inputs)['logits']
decoded =  np.argmax(output, axis=1)[0]

print('<REVIEW>')
print(review_example)
print()
print('<OUTPUT>')
print(decoded)

<REVIEW>
Its not the money,  it's the character of the team as a whole. Most of everyone is happy go luck here. There's grumbles anywhere I've gone to but grumbles here are 5inefficient things that you/we have the capacity to change. (Theses items are my only neutral or critical statements . I've been working since I was 8 yrs old and this is easily the best employer I've ever been able to earn with.
Thanks,to the owners, lower management , the greatest teams and HR. You've all been a blessing in the beginning of my young baby girl and immediate family's life and I am truly grateful for the opportunity to be here.

<OUTPUT>
4


<hr>

**RESUMEN**

In [32]:
task = 'summarization'
summarize = pipeline(task, model='knkarthick/MEETING_SUMMARY') # tuner007/pegasus_summarizer

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [33]:
print('<REVIEW>')
print(review_example)
print()
print('<OUTPUT>')
print(summarize(review_example))

<REVIEW>
Its not the money,  it's the character of the team as a whole. Most of everyone is happy go luck here. There's grumbles anywhere I've gone to but grumbles here are 5inefficient things that you/we have the capacity to change. (Theses items are my only neutral or critical statements . I've been working since I was 8 yrs old and this is easily the best employer I've ever been able to earn with.
Thanks,to the owners, lower management , the greatest teams and HR. You've all been a blessing in the beginning of my young baby girl and immediate family's life and I am truly grateful for the opportunity to be here.

<OUTPUT>
[{'summary_text': "This is easily the best employer I've ever been able to earn with. Its not the money, it's the character of the team."}]
